#TopicModeling Attemp 1


In [18]:
import os
os.chdir('drive/MyDrive/NLP/webtranscripts')

In [19]:
os.getcwd()

'/content/drive/MyDrive/NLP/webtranscripts'

In [20]:
# Let's read in our document-term matrix
import pandas as pd
import pickle

data = pd.read_pickle('dtm_stop.pkl')
data

,ability,able,absolutely,accelerated,acceptable,accepting,access,accessed,accessible,accessing,accidentally,accomplish,according,accordingly,account,accounting,accurate,accurately,achieve,achieving,acknowledge,acquire,acronym,act,action,actionable,actively,activity,actual,actually,ad,add,added,adding,addition,additional,additionally,address,addressed,adjust,...,wordstream,wordstreams,work,worked,working,world,worry,worse,worth,worthwhile,wouldnt,wp,wrexham,write,writer,writes,writing,written,writtent,wrong,wrote,www,wwwexamplecom,wwwneilpatelcom,xl,xml,xoxo,year,yearly,yes,yielded,yoast,york,youd,youll,yourappeal,youre,yoursitecomrobotstxt,youve,youwhats
brandpoint,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,1,1,0,1,2,1,0,1,2,0,1,1,1,0,0,0,...,0,0,1,1,0,0,0,0,2,0,0,0,0,0,0,0,3,3,0,0,2,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0
bruceclay,1,1,0,0,0,1,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,1,0,0,2,0,10,4,0,10,6,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,1,1,0,1,0,1,0
developers,0,1,0,2,0,0,2,1,1,1,1,0,1,0,0,1,1,2,0,0,1,0,1,0,1,0,0,0,1,1,1,6,4,6,2,3,0,0,1,0,...,0,0,3,0,1,1,0,0,0,0,1,0,0,4,0,0,5,2,0,1,0,2,1,0,0,2,0,1,0,0,0,0,0,3,2,0,18,0,5,0
jdrgroup,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1,2,0,0,0,0,0,0,0,0,0,0,...,0,0,4,0,0,0,2,0,0,0,0,0,1,2,0,0,4,4,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
neilpatel1,1,0,2,0,0,0,1,0,1,0,0,2,7,0,0,0,2,1,5,1,0,2,0,0,7,0,0,0,0,0,0,3,0,1,0,1,0,4,0,0,...,0,0,7,0,0,3,2,0,0,0,0,0,0,23,3,1,10,4,1,0,1,0,0,1,1,0,0,0,1,3,0,2,1,0,1,1,12,0,4,1
reliablesoft,0,2,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,8,1,0,0,0,1,2,1,0,0,0,0,0,3,0,1,0,0,0,1,0,3,...,0,0,0,0,1,0,0,0,0,0,0,0,0,2,0,0,9,1,0,0,1,0,0,0,0,0,0,2,0,1,0,0,0,0,6,0,5,0,0,0
wordstream,0,1,0,0,0,0,1,0,0,0,0,0,0,0,3,0,1,0,1,0,0,0,0,0,1,1,0,2,11,10,5,1,1,0,0,9,2,1,0,0,...,4,1,8,0,4,5,0,1,0,1,0,0,0,0,0,0,2,0,0,1,0,0,0,0,0,2,0,2,0,0,0,0,0,3,8,0,18,1,6,0


In [21]:
# Import the necessary modules for LDA with gensim
# Terminal / Anaconda Navigator: conda install -c conda-forge gensim
from gensim import matutils, models
import scipy.sparse


In [22]:
# One of the required inputs is a term-document matrix
tdm = data.transpose()
tdm.head()

,brandpoint,bruceclay,developers,jdrgroup,neilpatel1,reliablesoft,wordstream
ability,0,1,0,0,1,0,0
able,1,1,1,0,0,2,1
absolutely,0,0,0,0,2,0,0
accelerated,0,0,2,0,0,0,0
acceptable,0,0,0,1,0,0,0


In [23]:

# We're going to put the term-document matrix into a new gensim format, from df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [24]:
# Gensim also requires dictionary of the all terms and their respective location in the term-document matrix
cv = pickle.load(open("cv_stop.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

In [25]:
# Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term),
# we need to specify two other parameters as well - the number of topics and the number of passes
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)
lda.print_topics()

[(0,
  '0.009*"user" + 0.009*"link" + 0.006*"topic" + 0.006*"campaign" + 0.006*"website" + 0.006*"title" + 0.005*"example" + 0.005*"time" + 0.005*"business" + 0.005*"headline"'),
 (1,
  '0.012*"article" + 0.010*"traffic" + 0.007*"business" + 0.007*"tool" + 0.006*"term" + 0.006*"link" + 0.006*"like" + 0.005*"topic" + 0.005*"ranking" + 0.005*"website"')]

In [26]:
# LDA for num_topics = 3
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=10)
lda.print_topics()

[(0,
  '0.009*"traffic" + 0.009*"business" + 0.008*"ranking" + 0.007*"campaign" + 0.006*"tool" + 0.006*"marketing" + 0.006*"link" + 0.006*"headline" + 0.006*"dont" + 0.006*"time"'),
 (1,
  '0.017*"user" + 0.015*"link" + 0.010*"url" + 0.010*"avoid" + 0.009*"text" + 0.009*"website" + 0.007*"using" + 0.007*"image" + 0.007*"tag" + 0.006*"information"'),
 (2,
  '0.024*"article" + 0.007*"topic" + 0.006*"piece" + 0.006*"question" + 0.006*"quality" + 0.006*"key" + 0.006*"user" + 0.005*"website" + 0.005*"like" + 0.005*"writing"')]

In [27]:

# LDA for num_topics = 4
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=50)
lda.print_topics()

[(0,
  '0.016*"traffic" + 0.011*"business" + 0.010*"tool" + 0.009*"link" + 0.008*"term" + 0.008*"ranking" + 0.007*"look" + 0.007*"understand" + 0.007*"like" + 0.006*"youre"'),
 (1,
  '0.013*"user" + 0.013*"article" + 0.010*"link" + 0.010*"topic" + 0.009*"website" + 0.009*"campaign" + 0.007*"url" + 0.006*"avoid" + 0.006*"text" + 0.006*"example"'),
 (2,
  '0.010*"question" + 0.009*"article" + 0.007*"key" + 0.007*"piece" + 0.007*"factor" + 0.006*"answer" + 0.005*"comprehensive" + 0.005*"term" + 0.004*"featured" + 0.004*"backlinks"'),
 (3,
  '0.013*"headline" + 0.008*"copywriting" + 0.008*"write" + 0.007*"time" + 0.006*"best" + 0.006*"people" + 0.006*"target" + 0.006*"blog" + 0.006*"web" + 0.006*"useful"')]

#Topic Modeling Attemp 2 - With Nouns Only


In [28]:
# Let's create a function to pull out nouns from a string of text
from nltk import word_tokenize, pos_tag

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

In [29]:
# Read in the cleaned data, before the CountVectorizer step
data_clean = pd.read_pickle('/content/drive/MyDrive/NLP/webtranscripts/clean.pkl')
data_clean

,transcript
brandpoint,reviewing key factor creating highquality cont...
bruceclay,youve written article think pretty good fit co...
developers,manage monetize promote online content via goo...
jdrgroup,seo article opposed type written content cross...
neilpatel1,want traffic yes want trafficno enough traffic...
reliablesoft,alex comment one critical factor achieving lon...
wordstream,wordstream come known mostly ppc destination a...


In [30]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [31]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns = pd.DataFrame(data_clean.transcript.apply(nouns))
data_nouns

,transcript
brandpoint,factor highquality content audience search eng...
bruceclay,article compete article google index topic pos...
developers,manage content search guide owner business web...
jdrgroup,article crossover type article article visibil...
neilpatel1,traffic yes traffic blog postif team marketing...
reliablesoft,comment factor term success seo campaign matte...
wordstream,wordstream destination thing seo people time p...


In [32]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
                  'youre','youve', 'got', 'gonna', 'time', 'think', 'yeah', 'said']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words=stop_words)
data_cvn = cvn.fit_transform(data_nouns.transcript)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names())
data_dtmn.index = data_nouns.index
data_dtmn

,ability,access,account,accounting,accurate,achieve,action,activity,ad,add,addition,address,admire,advance,advantage,advertisement,advice,adwords,affiliate,afford,afloat,age,agency,ahrefs,aim,ainsworth,algorithm,align,allinone,allows,alt,analysis,analytics,analyze,analyzeexample,analyzer,analyzes,anchor,andor,angeles,...,webmaster,webpage,website,websitebusiness,week,weight,whats,wider,widget,wikipedia,window,winner,winter,wont,word,wordofmouth,wordpress,wordstream,wordstreams,work,world,worry,wp,wrexham,write,writer,writes,wwwexamplecom,wwwneilpatelcom,xml,xoxo,year,yes,yoast,york,youd,youll,yourappeal,yoursitecomrobotstxt,youwhats
brandpoint,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,2,0,0,0,0,0,0,0,0,0,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
bruceclay,1,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,0,0,1,0,1,1,...,1,3,0,0,0,0,1,0,0,0,0,0,0,0,5,1,3,0,0,0,1,0,1,0,2,4,0,0,0,0,1,0,0,0,0,1,1,0,0,0
developers,0,2,0,1,0,0,1,0,1,0,2,0,0,0,2,1,2,0,1,0,0,0,2,0,1,0,0,0,0,1,1,0,3,0,0,0,0,7,1,0,...,2,0,16,1,0,0,1,0,5,0,0,0,0,1,7,1,0,0,0,2,1,0,0,0,0,0,0,1,0,2,0,1,0,0,0,2,1,0,0,0
jdrgroup,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,4,0,2,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
neilpatel1,1,1,0,0,1,0,7,0,0,0,0,2,1,0,0,0,3,0,0,1,0,2,1,0,0,1,0,1,1,0,0,1,1,1,1,1,0,0,0,0,...,0,0,0,0,1,2,1,1,0,0,0,1,1,1,14,0,2,0,0,7,3,1,0,0,3,1,1,0,1,0,0,0,2,2,1,0,1,1,0,1
reliablesoft,0,0,3,0,0,1,2,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,...,1,0,3,0,0,0,0,0,0,0,0,0,0,1,5,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2,0,0,0,0,3,0,0,0
wordstream,0,1,3,0,1,0,1,2,5,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,4,0,0,0,1,0,6,0,0,0,0,1,0,0,...,3,0,1,0,0,0,2,0,3,1,1,0,0,2,2,0,0,2,1,7,5,0,0,0,0,0,0,0,0,2,0,2,0,0,0,2,7,0,1,0


In [33]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [34]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=50)
ldan.print_topics()

[(0,
  '0.033*"search" + 0.027*"seo" + 0.025*"content" + 0.022*"page" + 0.021*"google" + 0.020*"site" + 0.018*"keywords" + 0.016*"article" + 0.015*"engine" + 0.015*"keyword"'),
 (1,
  '0.058*"site" + 0.051*"page" + 0.039*"search" + 0.025*"google" + 0.020*"content" + 0.016*"use" + 0.015*"link" + 0.014*"result" + 0.014*"engine" + 0.011*"user"')]

In [35]:
# Let's try topics = 3
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=50)
ldan.print_topics()

[(0,
  '0.055*"article" + 0.027*"search" + 0.021*"google" + 0.020*"seo" + 0.019*"content" + 0.016*"keywords" + 0.012*"keyword" + 0.010*"engine" + 0.010*"business" + 0.009*"word"'),
 (1,
  '0.016*"question" + 0.016*"search" + 0.015*"article" + 0.015*"google" + 0.014*"post" + 0.013*"page" + 0.012*"result" + 0.012*"content" + 0.011*"factor" + 0.009*"piece"'),
 (2,
  '0.040*"site" + 0.039*"search" + 0.038*"page" + 0.027*"content" + 0.023*"google" + 0.022*"seo" + 0.018*"engine" + 0.015*"result" + 0.013*"keywords" + 0.012*"keyword"')]

In [36]:

# Let's try 4 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.035*"seo" + 0.035*"campaign" + 0.026*"content" + 0.020*"keywords" + 0.017*"marketing" + 0.017*"page" + 0.017*"article" + 0.016*"search" + 0.015*"step" + 0.014*"google"'),
 (1,
  '0.046*"search" + 0.045*"site" + 0.030*"page" + 0.027*"engine" + 0.025*"traffic" + 0.019*"google" + 0.019*"content" + 0.018*"seo" + 0.016*"business" + 0.014*"tool"'),
 (2,
  '0.040*"site" + 0.039*"search" + 0.038*"page" + 0.027*"google" + 0.027*"content" + 0.019*"seo" + 0.016*"engine" + 0.015*"result" + 0.014*"use" + 0.014*"article"'),
 (3,
  '0.018*"search" + 0.018*"question" + 0.017*"article" + 0.017*"google" + 0.016*"post" + 0.014*"page" + 0.014*"content" + 0.013*"result" + 0.012*"factor" + 0.010*"piece"')]

##Seems like with nouns only, the text seems meaning less. Lets try with Adjectives and Noun Together


In [37]:
# Let's create a function to pull out nouns from a string of text
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

In [38]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns_adj = pd.DataFrame(data_clean.transcript.apply(nouns_adj))
data_nouns_adj

,transcript
brandpoint,key factor highquality content audience search...
bruceclay,article good fit compete article google index ...
developers,manage promote online content google search gu...
jdrgroup,seo article type content crossover type articl...
neilpatel1,want traffic yes trafficno traffic blog postif...
reliablesoft,alex comment critical factor long term success...
wordstream,wordstream ppc destination thing seo people u ...


In [39]:
# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
cvna = CountVectorizer(stop_words=stop_words, max_df=.8)
data_cvna = cvna.fit_transform(data_nouns_adj.transcript)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names())
data_dtmna.index = data_nouns_adj.index
data_dtmna

,ability,able,acceptable,access,accessible,accomplish,account,accounting,accurate,achieve,acronym,action,actionable,activity,actual,ad,add,added,addition,additional,address,admire,advance,advanced,advantage,advertisement,advice,adwords,affect,affiliate,afford,afloat,age,agency,aggressive,agree,ahrefs,aim,ainsworth,alert,...,wide,wider,widget,wikipedia,willing,window,winner,winter,wish,wont,wordofmouth,wordpress,wordstream,wordstreams,work,world,worry,worse,worth,wouldnt,wp,wrexham,write,writer,writes,writtent,wrong,www,wwwexamplecom,wwwneilpatelcom,xml,xoxo,year,yes,yoast,york,youd,yourappeal,yoursitecomrobotstxt,youwhats
brandpoint,0,1,0,0,0,0,0,0,1,0,0,1,1,1,2,0,1,0,1,1,0,0,0,0,0,0,0,3,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
bruceclay,1,1,0,1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,1,4,0,0,0,1,0,0,0,0,1,0,6,4,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0
developers,0,1,0,2,1,0,0,1,1,0,1,1,0,0,1,1,1,0,2,3,0,0,0,0,2,3,2,0,1,1,0,0,0,2,0,0,0,1,0,1,...,0,0,6,0,1,0,0,0,1,2,1,0,0,0,2,1,0,0,0,1,0,0,3,0,0,0,1,1,1,0,2,0,1,0,0,0,2,0,0,0
jdrgroup,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,2,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
neilpatel1,1,0,0,1,1,2,0,0,2,0,0,7,0,0,0,0,0,0,0,1,4,1,0,0,0,1,3,0,0,0,1,0,2,1,0,0,0,0,1,0,...,0,1,0,0,0,0,1,1,0,2,0,2,0,0,7,3,1,0,0,0,0,0,15,3,1,1,0,0,0,1,0,0,0,2,2,1,0,1,0,1
reliablesoft,0,2,0,0,0,0,3,0,0,2,0,2,1,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,3,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0
wordstream,0,1,0,1,0,0,3,0,1,0,0,1,1,2,11,5,0,1,0,9,1,0,0,1,1,0,0,1,1,0,0,0,0,0,3,0,1,0,0,0,...,0,0,4,1,0,1,0,0,0,3,0,0,3,1,7,5,0,1,0,0,0,0,0,0,0,0,1,0,0,0,2,0,2,0,0,0,3,0,1,0


In [40]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [41]:

# Let's start with 2 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.057*"site" + 0.014*"link" + 0.011*"url" + 0.010*"tool" + 0.008*"text" + 0.008*"tag" + 0.008*"different" + 0.008*"avoid" + 0.007*"image" + 0.006*"rank"'),
 (1,
  '0.014*"campaign" + 0.011*"headline" + 0.011*"marketing" + 0.009*"site" + 0.007*"rank" + 0.006*"question" + 0.005*"facebook" + 0.005*"goal" + 0.005*"write" + 0.005*"tag"')]

In [42]:
# Let's try 3 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.054*"site" + 0.016*"link" + 0.012*"campaign" + 0.012*"url" + 0.010*"tool" + 0.009*"different" + 0.008*"text" + 0.008*"tag" + 0.008*"rank" + 0.008*"image"'),
 (1,
  '0.015*"site" + 0.007*"write" + 0.007*"clay" + 0.006*"tool" + 0.006*"plugin" + 0.006*"expertise" + 0.006*"optimization" + 0.006*"starter" + 0.005*"free" + 0.005*"key"'),
 (2,
  '0.015*"headline" + 0.011*"site" + 0.008*"marketing" + 0.008*"question" + 0.006*"rank" + 0.006*"tag" + 0.006*"brand" + 0.006*"useful" + 0.006*"web" + 0.006*"meta"')]

In [43]:
# Let's try 4 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=50)
ldana.print_topics()

[(0,
  '0.001*"site" + 0.001*"tool" + 0.001*"headline" + 0.001*"meta" + 0.001*"rank" + 0.001*"description" + 0.001*"link" + 0.001*"marketing" + 0.001*"tag" + 0.001*"type"'),
 (1,
  '0.029*"campaign" + 0.016*"marketing" + 0.010*"question" + 0.010*"facebook" + 0.007*"goal" + 0.006*"key" + 0.006*"rank" + 0.006*"link" + 0.005*"start" + 0.005*"publish"'),
 (2,
  '0.001*"site" + 0.001*"rank" + 0.001*"tool" + 0.001*"marketing" + 0.001*"campaign" + 0.001*"link" + 0.001*"work" + 0.001*"question" + 0.001*"type" + 0.001*"write"'),
 (3,
  '0.050*"site" + 0.012*"link" + 0.010*"tool" + 0.009*"tag" + 0.009*"url" + 0.009*"headline" + 0.008*"rank" + 0.007*"description" + 0.007*"meta" + 0.007*"avoid"')]

#We can see that it shows best result at 3 clusters, so we will keep it to 3 clusters

#Lets Identify Topics from each document 

In [65]:
# Our final LDA model (for now)
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=80)
topics = ldana.print_topics()
topics

[(0,
  '0.015*"site" + 0.007*"write" + 0.007*"clay" + 0.006*"expertise" + 0.006*"optimization" + 0.006*"plugin" + 0.006*"starter" + 0.006*"tool" + 0.005*"guideline" + 0.005*"point"'),
 (1,
  '0.054*"site" + 0.016*"link" + 0.012*"campaign" + 0.012*"url" + 0.010*"tool" + 0.009*"different" + 0.008*"text" + 0.008*"rank" + 0.008*"tag" + 0.008*"image"'),
 (2,
  '0.015*"headline" + 0.011*"site" + 0.009*"marketing" + 0.008*"question" + 0.006*"brand" + 0.006*"web" + 0.006*"useful" + 0.006*"rank" + 0.006*"tag" + 0.006*"audience"')]

In [131]:
#Let's extact the single words from the list of topic and their cluster
import re
match_str = [[] for j in range(len(topics))]

match_words = ['']*len(topics)
for i in range(len(topics)):
  match_words[i] = re.findall("\"[a-z]*\"",topics[i][1]  ) 
  for j in (match_words[i]):
    match_str[i].append(re.findall("[a-z]*",j)[1])
  print(match_str[i])
match_str

['site', 'write', 'clay', 'expertise', 'optimization', 'plugin', 'starter', 'tool', 'guideline', 'point']
['site', 'link', 'campaign', 'url', 'tool', 'different', 'text', 'rank', 'tag', 'image']
['headline', 'site', 'marketing', 'question', 'brand', 'web', 'useful', 'rank', 'tag', 'audience']


In [134]:
# Let's take a look at which topics each transcript contains
corpus_transformed = ldana[corpusna]
list_topics =  list(zip([a for [(a,b)] in corpus_transformed], data_dtmna.index))
list_topics

[(2, 'brandpoint'),
 (0, 'bruceclay'),
 (1, 'developers'),
 (2, 'jdrgroup'),
 (2, 'neilpatel1'),
 (1, 'reliablesoft'),
 (1, 'wordstream')]

In [52]:
#installing the module 
import spacy.cli
spacy.cli.download("en_core_web_md")

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [54]:
# importing the library and loding the vectors in the nlp
import spacy
  
nlp = spacy.load('en_core_web_md')

Enter two space-separated words
SEO search
SEO True 7.7217484 False
search True 6.2328997 False
Similarity: 0.46332645


In [ ]:
# Let's calculate the similarity of our searched word with list of topics we recieved after LDA
match = [0]*len(topics)
match_word = "SEO"

k = 0

for i in match_str:
  for j in i:
    words = j +" "+ match_word
    #print(words)
    tokens = nlp(words)
    token1, token2 = tokens[0], tokens[1]
    match[k] = match[k] + token1.similarity(token2)
  k = k + 1
print(match)
    # print("Similarity:", token1.similarity(token2))

In [136]:
#match which cluster matched the most
max_match = 0
at_index = 0
k = 0
for i in match:
  if max_match < i:
    max_match = i
    at_index = k
  k = k + 1
at_index

2

In [137]:
#Printing the websites that match the most
for i in list_topics:
  if i[0] == at_index:
    print(i[1])

brandpoint
jdrgroup
neilpatel1
